In [ ]:
#https://deap.readthedocs.io/en/master/examples/gp_symbreg.html

import operator
import math
import random

import numpy as np
np.seterr(all = "raise")
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import matplotlib.pyplot as plt

%matplotlib inline

# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except (ZeroDivisionError,ValueError,FloatingPointError):
        return 1.0

def protectedlog(value):
    try:
        return np.log(value)
    except (ValueError,FloatingPointError):
        return 1.0

pset = gp.PrimitiveSet("MAIN", 1)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
#pset.addPrimitive(operator.neg, 1)
pset.addPrimitive(np.cos, 1)
pset.addPrimitive(np.sin, 1)
pset.addPrimitive(np.exp, 1)
pset.addPrimitive(protectedlog, 1)
#pset.addEphemeralConstant("rand105", lambda: random.uniform(-10,10))
pset.renameArguments(ARG0='x')

creator.create("FitnessMin", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=5)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalSymbReg(individual, points):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    # Evaluate the mean squared error between the expression
    # and the real function : x**4 + x**3 + x**2 + x
    sqerrors = 0
    try:
        for x in points:
            sqerrors += np.abs(func(x[0]) - x[1])
    except (ZeroDivisionError,ValueError,FloatingPointError):
        #print(individual)
        #print("x[0] = ",x[0])
        #print("x[1] = ",x[1])
        #print("sqerrors = ",sqerrors)
        print("exception!")
        sqerrors = 100000

    return (-1.0*sqerrors,)

toolbox.register("evaluate", evalSymbReg, points=np.array(
    [[-1.0, 0.0000],[-0.9, -0.1629],[-0.8, -0.2624],[-0.7, -0.3129],
    [-0.6, -0.3264],[-0.5, -0.3125],[-0.4, -0.2784],[-0.3, -0.2289],
    [-0.2, -0.1664],[-0.1, -0.0909],[0.0, 0.0],[0.1, 0.1111],
    [0.2, 0.2496],[0.3, 0.4251],[0.4, 0.6496],[0.5, 0.9375],
    [0.6, 1.3056],[0.7, 1.7731],[0.8, 2.3616],[0.9, 3.0951],[1.0, 4.0000]]))
toolbox.register("select", tools.selTournament, tournsize=800)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=8))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=5))

def main():
    random.seed(12)
    
    

    pop = toolbox.population(n=1000)
    hof = tools.HallOfFame(1)

    stats_fit = tools.Statistics(lambda ind: ind.fitness.values)
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", np.mean)
    mstats.register("std", np.std)
    mstats.register("min", np.min)
    mstats.register("max", np.max)
    

    pop, log = algorithms.eaSimple(pop, toolbox, 0.7, 0.0, 50, stats=mstats,
                                   halloffame=hof, verbose=False)
    # print log
    return pop, log, hof


pop, log, hof = main()
print(hof[0])

In [ ]:
min_log,max_log = log.chapters["fitness"].select("min","max")
min_size_log,max_size_log,avg_size_log = log.chapters["size"].select("min","max","avg")

f = plt.figure()
f.set_figwidth(12)
f.set_figheight(8)
plt.plot(max_log)
plt.xlabel("generation")
plt.ylabel("best individual fitness")
plt.show()

f = plt.figure()
f.set_figwidth(12)
f.set_figheight(8)
plt.plot(min_size_log)
plt.xlabel("generation")
plt.ylabel("size best individual")
plt.show()

f = plt.figure()
f.set_figwidth(12)
f.set_figheight(8)
plt.plot(avg_size_log)
plt.xlabel("generation")
plt.ylabel("average size")
plt.show()

f = plt.figure()
f.set_figwidth(12)
f.set_figheight(8)
plt.bar(range(51),max_size_log,label="max size")
plt.bar(range(51),min_size_log,label="min size")
plt.xlabel("generation")
plt.ylabel("size")
plt.legend()
plt.show()

In [ ]:
points=np.array(
    [[-1.0, 0.0000],[-0.9, -0.1629],[-0.8, -0.2624],[-0.7, -0.3129],
    [-0.6, -0.3264],[-0.5, -0.3125],[-0.4, -0.2784],[-0.3, -0.2289],
    [-0.2, -0.1664],[-0.1, -0.0909],[0.0, 0.0],[0.1, 0.1111],
    [0.2, 0.2496],[0.3, 0.4251],[0.4, 0.6496],[0.5, 0.9375],
    [0.6, 1.3056],[0.7, 1.7731],[0.8, 2.3616],[0.9, 3.0951],[1.0, 4.0000]])

f = plt.figure()
f.set_figwidth(12)
f.set_figheight(8)
func2 = toolbox.compile(expr=hof[0])

def func(x):
    return x*np.exp(x)

plt.plot(points[:,0],points[:,1],label = "true values",linestyle = "", marker = "o")
plt.plot(points[:,0],func(points[:,0]),label = "learned function")
plt.plot(points[:,0],func2(points[:,0]),label = "complex learned function")
plt.legend()
plt.show()